**To Do:**

- REFACTOR, Clean up code considerably!
  -  separate csv and json saving methods.


In [ ]:
!pip install selenium # in package file

     |████████████████████████████████| 958 kB 14.8 MB/s 
     |████████████████████████████████| 138 kB 22.6 MB/s 
     |████████████████████████████████| 356 kB 45.8 MB/s 
     |████████████████████████████████| 3.6 MB 38.1 MB/s 
     |████████████████████████████████| 55 kB 3.1 MB/s 
     |████████████████████████████████| 54 kB 3.2 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.8 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [ ]:
#imports
from bs4 import BeautifulSoup
from datetime import datetime as dt
from google.colab import drive
from selenium.webdriver.support.ui import WebDriverWait       
from selenium.webdriver.common.by import By       
from selenium.webdriver.support import expected_conditions as EC
from urllib import request
from urllib.request import urlopen
import csv
import io
import json
import os
import random
import re
import time   # to slow down the time between requests
              # to minimize the risk of ip addresses being blocked.

In [ ]:

!apt-get update
!apt install chromium-chromedriver

from selenium import webdriver
from selenium.webdriver.chrome.options import Options

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:5 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:7 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:8 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Packages [2,489 kB]
Hit:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Get:10 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packages [1,459 kB]
Get:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Get:12 http://security.ubuntu.com/ubuntu bionic-security/restricted amd64 Packages [716 kB]
Get:13 http://security.ubuntu.com/ubuntu bio

In [ ]:
drive.mount('/content/gdrive')
ROOT = 'gdrive/MyDrive/Austin_Drinks'
ROOT_IMPORTS = 'gdrive/MyDrive/Colab Notebooks/Austin_Drinks_Package';

Mounted at /content/gdrive


SITES is a dictionary containing attributes such as url, attributes, attribute definitions, and search extensions

If these items on the website changes, then it will be a simple update in the SITES dictionary.

In [ ]:
SITES =   { 
            'yelp' :  {
                        # base url for yelp
                        'url':'https://www.yelp.com',
                        # header containing the anchor tag we need for url's extention to each establishment page
                        'search_results_tags' : 'h4',
                       
                        # search_results_tags 
                        # this class is associated with distilleries, breweries and wineries search results
                        # thus, this class should work for all establishment types.
                        # the h4 class containing the anchor tags we need for url's extension to each establishment page
                        'search_results_tags_class' : 'css-1l5lt1i',
                       
                        # attributes of button to reveal accommodations and unaccommodations
                        'amenities_reveal_button_text' : 'Attribute',
                        'amenities_reveal_button_attr' : 'aria-controls',
                        'amenities_reveal_button_attr2' : 'class',
                        'amenities_reveal_button_attr2_def' : ' css-174jypu',
                       
                       # tag information to get the json data
                        'json_data_tag' : 'div',
                        'json_data_tag_attr' : 'class',
                        'json_data_tag_attr_def' : 'main-content-wrap main-content-wrap--full',
                       
                       # tag information to get businesses url
                        'business_url_tag' : 'div',
                        'business_url_tag_attr' : 'class',
                        'business_url_tag_attr_def' : ' css-1vhakgw border--top__373c0__2SDyx border-color--default__373c0__1WSID',
                       
                       # tag information to get establishment's tags
                        'est_tags_tag' : 'span',
                        'est_tags_tag_attr' : 'class',
                        'est_tags_tag_attr_def' : ' display--inline__373c0__3d-lf margin-r1__373c0__7ZINV border-color--default__373c0__2s5dW',
                        'est_tags_sub_tag_attr' : 'a',
                       
                       # hours of operation table tag information
                        'hoO_tag' : 'table',
                        'hoO_tag_attr' : 'class',
                        'hoO_tag_attr_def' : ' hours-table__373c0__2YHlD table__373c0__1FIZ8 table--simple__373c0__3QsR_',
                        'hoO_row_tag_attr' : 'class',
                        'hoO_row_tag_attr_def' : ' table-row__373c0__1F6B0',
                       
                       # amenities and anti-amenities tag information
                        'amenities_tag' : 'span', 
                        'amenities_tag_attr' : 'class', 
                        'amenities_tag_attr_def' : ' css-1h1j0y3', 
                        'anti_amenities_tag' : 'span', 
                        'anti_amenities_tag_attr' : 'class', 
                        'anti_amenities_tag_attr_def' : ' css-1h1j0y3', 
                       
                        #different extensions to be used for yelp.com.
                        'austin_distillery_search_ext': '/search?find_desc=distilleries&find_loc=Austin%2C+TX',
                        'austin_winery_search_ext' : '/search?find_desc=wineries&find_loc=Austin%2C%20TX',
                        'austin_brewery_search_ext' : '/search?find_desc=breweries&find_loc=Austin%2C%20TX',
                       #add methods to save locations, use datetime to track when information needs to be updated (think permanently closed establishments)
                      }
           }

Notes/References on class creation

In [ ]:
# using type function to create classes on the fly.

# est_obj = type('est_class', (object,), { 'variable_1': value_1, 'variable_2': value_2, ... , 'variable_n': value_n })()

In [ ]:
# or use a class definition

# class est_class():
#   def __init__(self, value_1, value_2, ... , value_n):
#     self.variable_1 = value_1
#     self.variable_2 = value_2
#     ...
#     self.variable_n = value_n

Used to log errors that occur during the scraping process

In [ ]:
def __errorLog(link, error):
  with open(ROOT + '/Data_Collecting/error_log.txt', 'a') as f:
    f.write('{}\t\t{}\t\t{}\n'.format(dt.now(), link, error))

Utility functions to save and load JSON files

In [ ]:
# save JSON data

def saveJSON(data):  
  with io.open(ROOT + '/Data_Collecting/JSON files/' + data['name'] + '.json', 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii = False, indent = 4)

In [ ]:
def loadJSON(name):  
  with io.open(ROOT + '/Data_Collecting/JSON files/' + name, 'r', encoding='utf-8') as f:
    data = json.load(f)
  return data

Utility functions to save, append, and update CSV files.

In [ ]:
def saveNewCSV(data):
  with io.open(ROOT + )

Function to clean names up, replace unwanted symbols such as '&' and such.

In [ ]:
def cleanName(name):
  cleaned = re.sub('&amp;', 'and', name)
  cleaned = re.sub('&apos;', '\'', cleaned)
  return cleaned

Use urllib to get a url's html and return a beautifulsoup object

In [ ]:
# function specifically designed to find and click a button.

def getBS_via_selenium(url, onclickButton = False, in_text = '', attr = ''):
  chrome_options = Options()
  chrome_options.add_argument('--headless') # don't spawn window
  chrome_options.add_argument('--no-sandbox')
  chrome_options.add_argument('--disable-dev-shm-usage')
  chrome_options.add_argument('user-agent = Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36')
  driver = webdriver.Chrome(executable_path = '/usr/lib/chromium-browser/chromedriver', options = chrome_options)
  print('Compiling BS object at url @ ' + url)

  # first, get the page where the button we want is at
  source = None
  attempts = 0
  max_attempts = 4
  unsuccessful = True 
  while unsuccessful:
    attempts += 1
    try:
      print('Getting url via selenium')
      driver.get(url)
      time.sleep(random.randint(10,14))
    except BaseException as ex:
      if attempts < max_attempts:
        print('Unable to open url. Retrying in 30 seconds. \
               See exception repr for details: ')
        print(repr(ex))      # print object as string
        time.sleep(30)
      else:
        print('Too many attepts, breaking loop')
        #log error to file.
        __errorLog(url, 'Too many attempts')
    else:
      unsuccessful = False
      source = BeautifulSoup(driver.page_source, 'html.parser')

    
      # second, if we are interested in clicking a button,
      # we repeat the process using the existing driver to click the button
      # and extract the data

      if onclickButton:
          attempts = 0
          max_attempts = 2
          unsuccessful = True 
          while unsuccessful:
            attempts += 1
            try:
              # this aria-controls is unique to the button we are looking for..
 #             buttons = source.find_all('button', {'class': ' css-174jypu'})  # find buttons with class of interest
              attr_def = ''
              for button in source.find_all('button', {'class': ' css-174jypu'}):
                if in_text in button.text:
                  in_text_found = True
                  attr_def = button[attr]

#                  <button id="onetrust-pc-btn-handler" tabindex="0"> is blocking my ability to click a button now, wtf..
                  driver.execute_script("arguments[0].click();", WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'button[' + attr + '=' + attr_def + ']'))))
#                  WebDriverWait(driver, 20).until(EC.invisibility_of_element((By.CSS_SELECTOR, 'button[' + attr + '=' + attr_def + ']'))).click()
#                  driver.execute_script("arguments[0].click();", WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button." + attr_def))))

#                  element = driver.find_element_by_css_selector('button[' + attr + '=' + attr_def + ']')
#                  element.click()
                  print('clicked element')
                  time.sleep(random.randint(3,6))
                  break
            except BaseException as ex:
              if attempts < max_attempts:
                print('Unable to click button. Retrying in 30 seconds. \
                      See exception repr for details: ')
                print(repr(ex))      # print object as string
                time.sleep(30)
              else:
                print('Too many attempts to click button, breaking loop')
                #log error to file.
                __errorLog(url, 'Exceeded click button request limit to extract Amenities')
            else:
              unsuccessful = False             
              source = BeautifulSoup(driver.page_source, 'html.parser')              

  driver.close()
  driver.quit()
  return source

In [ ]:
# takes a url as input (must be a complete url as string) and returns a BS object.

def getBS_urllib(url):
  req = request.Request(
      url,
      headers = {
          'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36'
          }
      )
  print('Compiling BS object at url @ ' + url)
  attempts = 0
  max_attempts = 4
  html = None
  bs = None
  unsuccessful = True 
  while unsuccessful:
    attempts += 1
    try:
      html = urlopen(req)
      time.sleep(random.randint(3,8))
    except BaseException as ex:
      if attempts < max_attempts:
        print('Unable to open url. Retrying in 30 seconds.  See exception repr for details:')
        print(repr(ex))      # print object as string
        time.sleep(30)
      else:
        print('Too many attepts, breaking loop')
        #log error to file.
        __errorLog(url, 'Exceeded url request limit')
        break
    else:
      unsuccessful = False
      bs = BeautifulSoup(html.read(), 'html.parser')
  return bs

Define a function that returns a list of search result links

In [ ]:
# currently, this may only work for yelp.com.
# trip advisor for sure is a different story.

def get_yelp_search_result_links(url_curr, lst):

  print('Gathering search result urls @ ' + url_curr)

  bs = getBS_urllib(url_curr)


  h4_tag = bs.find_all(SITES['yelp']['search_results_tags'], class_= SITES['yelp']['search_results_tags_class'])
  #h4_tag = bs.find_all('h4', {'class':'css-1l5lt1i'}) # this equivalent to the line above.
  #print(h4_tag)
  #for h in h4_tag:
  #  print(h.prettify())
  for h in h4_tag:
    if h.find('a')['href'][0:9] != '/adredir?':
      lst.append(SITES['yelp']['url'] + h.find('a')['href'])

  # get the next page anchor on current page, then go to the next page recursively until there are no more pages.
  next_page = bs.find('a', {'class' : 'next-link navigation-button__09f24__1EzzD css-166la90'})
  if next_page != None:
    get_yelp_search_result_links(next_page['href'], lst)

  # end method

the next set of cells collects establishment links and saves them to a file to load later.

In [ ]:
distillery_url_list = []    
print('Gathering "Distilleries" search results ')
get_yelp_search_result_links(SITES['yelp']['url'] + SITES['yelp']['austin_distillery_search_ext'], distillery_url_list)
for url in distillery_url_list:
  print('Found link to ' + url)
print('Number of Distillery urls: ' + str(len(distillery_url_list)))

Gathering "Distilleries" search results 
Gathering search result urls @ https://www.yelp.com/search?find_desc=distilleries&find_loc=Austin%2C+TX
Compiling BS object at url @ https://www.yelp.com/search?find_desc=distilleries&find_loc=Austin%2C+TX
Number of Distillery urls: 0


In [ ]:
with open(ROOT + '/Data_Collecting/distillery_links.txt', 'w', encoding='utf-8') as f:
  for url in distillery_url_list:
    f.write(url + '\n')

FileNotFoundError: ignored

In [ ]:
winery_url_list = []    
print('Gathering "Wineries" search results ')
get_yelp_search_result_links(SITES['yelp']['url'] + SITES['yelp']['austin_winery_search_ext'], winery_url_list)
for url in winery_url_list:
  print('Found link to ' + url)
print('Number of Winery urls: ' + str(len(winery_url_list)))

In [ ]:
with open(ROOT + '/Data_Collecting/winery_links.txt', 'w', encoding='utf-8') as f:
  for url in winery_url_list:
    f.write(url + '\n')

In [ ]:
brewery_url_list = []    
print('Gathering "Breweries" search results ')
get_yelp_search_result_links(SITES['yelp']['url'] + SITES['yelp']['austin_brewery_search_ext'], brewery_url_list)
for url in brewery_url_list:
  print('Found link to ' + url)
print('Number of Brewery urls: ' + str(len(brewery_url_list)))

In [ ]:
with open(ROOT + '/Data_Collecting/brewery_links.txt', 'w', encoding='utf-8') as f:
  for url in brewery_url_list:
    f.write(url + '\n')

Cells to load links from file

In [ ]:
distillery_url_list = []
winery_url_list = []   
brewery_url_list = []   

with open(ROOT + '/Data_Collecting/distillery_links.txt', 'r', encoding='utf-8') as f:
  for line in f.readlines():
    distillery_url_list.append(line)

with open(ROOT + '/Data_Collecting/winery_links.txt', 'r', encoding='utf-8') as f:
  for line in f.readlines():
    winery_url_list.append(line)

with open(ROOT + '/Data_Collecting/brewery_links.txt', 'r', encoding='utf-8') as f:
  for line in f.readlines():
    brewery_url_list.append(line)

print('distilleries:  ' + str(len(distillery_url_list)))
print('wineries:      ' + str(len(winery_url_list)))
print('breweries:     ' + str(len(brewery_url_list)))

Function that retrieves an establishment's information from yelp via JSON and the page itself.

TODO Separate out to only collect JSON data
Make a separate function to save into csv files


In [ ]:
# instead of collecting a list of htmls for each link,
# it might be better to save space and collect information
# we need from each site one at a time,
# then offload it into a file or DB somewhere..

def get_yelp_establishment_information(link, infoCSV, infoJSON, SITES):

  print('Attempting to collect from ' + link)

  html = getBS_via_selenium(link, 
                            onclickButton = True, 
                            button_text = SITES['yelp']['amenities_reveal_button_text'], 
                            hider_attr = SITES['yelp']['amenities_reveal_button_attr'], 
                            button_attr = SITES['yelp']['amenities_reveal_button_attr2'], 
                            button_attr_def = SITES['yelp']['amenities_reveal_button_attr2_def'])
  
  # what information do we want? json script tags may be useful here, too

  # script tag we want: div class = 'main-content-wrap main-content-wrap--full' only one of these tags

    #                                  main-content-wrap main-content-wrap--full
  div_tag = html.find('div', class_ = 'main-content-wrap main-content-wrap--full')
  script_tags = div_tag.find_all('script', {'type' : 'application/ld+json'})

  #print(len(script_tags))
  #for st in script_tags:
  #  print(st.prettify())
  # looks like it is the second script_tag with json information we want.  


  # if a json is available on the web page, we get that json and store it here
  # we collect whatever information may be available in that json.
  infoJSON = None
  try:
    # strict omits control characters when True.. or change to false and re.sub with ' ' instead.
    infoJSON = json.loads(''.join(script_tags[1].contents), strict = True)
  except:
    print('Unable to get data from json, saving failed link and moving forward..')
    __errorLog(link, 'JSON data not found')
  else:
    # extract info in JSON
    # name of establishment

    name = ''
    if 'name' in infoJSON:
      infoJSON['name'] = cleanName(infoJSON['name'])
      name = infoJSON['name']
      infoCSV.append(infoJSON['name'])
      print('Collecting information for ' + name)
    else:
      print('No name for url ' + link)
      infoCSV.append('unnamed establishment')
      __errorLog(link, 'JSON data has no \'name\' key')

    # address
    if 'address' in infoJSON and 'streetAddress' in infoJSON['address']:
      streetAddress = re.sub('\n', '', infoJSON['address']['streetAddress'])
      infoCSV.append(streetAddress)
    else:
      print('No street address available for ' + name)
      infoCSV.append('')

    if 'address' in infoJSON and 'addressLocality' in infoJSON['address']:
      localAddress = infoJSON['address']['addressLocality']
      infoCSV.append(localAddress)
    else:
      print('No local address available for ' + name)
      infoCSV.append('')

    if 'address' in infoJSON and 'addressRegion' in infoJSON['address']:
      regionAddress = infoJSON['address']['addressRegion']
  #   print(regionAddress)
      infoCSV.append(regionAddress)
    else:
      print('No region address available for ' + name)
      infoCSV.append('')

    if 'address' in infoJSON and 'postalCode' in infoJSON['address']:
      postalCode = infoJSON['address']['postalCode']
  #   print(postalCode)
      infoCSV.append(postalCode)
    else:
      print('No postal code available for ' + name)
      infoCSV.append('')
    
    # phone number
    if 'telephone' in infoJSON:
      telephone = infoJSON['telephone']
  #   print(telephone)
      infoCSV.append(telephone)
    else:
      print('No telephone available for ' + name)
      infoCSV.append('')

    # dollar signs/price range
    if 'priceRange' in infoJSON:
      priceRange = infoJSON['priceRange']
  #   print('price range: ' + priceRange)
      infoCSV.append(priceRange)
    else:
      print('No price range available for ' + name)
      infoCSV.append('')

    # review rating
    if 'aggregateRating' in infoJSON and 'ratingValue' in infoJSON['aggregateRating']:
      ratingValue = infoJSON['aggregateRating']['ratingValue']
  #   print('rating value: ' + str(ratingValue))
      infoCSV.append(ratingValue)
    else:
      print('No rating value available for ' + name)
      infoCSV.append('')

    # number of reviews
    if 'aggregateRating' in infoJSON and 'reviewCount' in infoJSON['aggregateRating']:
      ratingCount = infoJSON['aggregateRating']['reviewCount']
  #   print('rating count: ' + str(ratingCount))
      infoCSV.append(ratingCount)
    else:
      print('No rating count available for ' + name)
      infoCSV.append('')

    # website

    # <div class = ' css-1vhakgw border--top__373c0__19Owr border-color--default__373c0__2oFDT'>
      # <a>
    div_tag = html.find('div', class_ = ' css-1vhakgw border--top__373c0__2SDyx border-color--default__373c0__1WSID')
    if div_tag.find('a'):
      website = div_tag.find('a').get_text()
      infoCSV.append(website)
      infoJSON['website'] = website
    else:
      print('No website available for ' + name)
      infoCSV.append('')
      
    # est. type.
    span_tags = html.find_all('span', {'class' : ' display--inline__373c0__3d-lf margin-r1__373c0__7ZINV border-color--default__373c0__2s5dW'})
    tags = []
    for span in span_tags:
      anc_tags = span.find_all('a')
  #    print(str(len(anc_tags)))   # there could be zero or more anchor tags to get text from..
      if anc_tags:
        est_str = ''
        for anc in anc_tags:
          if anc.get_text() != 'Unclaimed':
            est_str += anc.get_text() + '|'    # cannot use commas as a delimiter because we are currently saving to csv format.
            tags.append(anc.get_text())
        if est_str:
          infoCSV.append(est_str)
  #        print(est_str)
        else:
          print('No type string available for ' + name)

    infoJSON['tags'] = tags

    # hours of operations, if available  
      # this is specifically for yelp.com.. so move to the dictionary for yelp.com once it is working.
      # <table class = ' hours-table__373c0__1S9Q_ table__373c0__1paZL table--simple__373c0__3hEOO' ...>
      # yes there is a space at the front
        # <tr class = ' table-row__373c0__3xT3x' ...>
          # <th class = ' table-header-cell__373c0__OywTx' ...> gets the day of the week
          # <td class = ' table-cell__373c0__Hc_7A table-cell--top__373c0__1hgmO' ...> gets the hours of the day
          # and td of same class above also gets for current day "Open", "Closed now", etc.. if text is present, blank otherwise.

    data = {}
    table = html.find('table', {'class' : ' hours-table__373c0__2YHlD table__373c0__1FIZ8 table--simple__373c0__3QsR_'})
    if table.find('tbody').contents:
      table_row = table.find_all('tr', {'class' : ' table-row__373c0__1F6B0'})
  #    arr = []
      day = ''
      hours = ''
      for row in table_row:
        day_str = ''
        hours_str = ''
        # get th and td
        if row.find('th'):
          day_str = row.find('th').get_text()
          day += day_str + '|'
        else:
          day += '|'
        if row.find('td'):
          hours_str = row.find('td').get_text()
          hours += hours_str + '|'
        else:
          hours += '|'
        
        data[day_str] = hours_str
      infoJSON['Hours'] = data
      infoCSV.append(day)
      infoCSV.append(hours)
  #      a = []
  #      a.append(day)   # 1 column
  #      a.append(hours)   # 1 column, only getting the first of two.
  #      arr.append(a)  
  #    data = pd.DataFrame(arr, columns = ['Day', 'Hours'])
  #    print(data)
    else:
      print('No hours table is available for ' + name) 
      # empty columns for 'day' and 'hours'.
      for i in range(0, 2): 
        infoCSV.append('||||||')


    # Amenities, if it exists.
    if 'Amenities and More' in html.text:
      amenities_tags = html.find_all('span', {'class': ' css-1h1j0y3'})     # what the company accommodates
      anti_amenities_tags = html.find_all('span', {'class': ' css-chtywg'}) # what the company does not accommodate
      infoJSON['Accommodations'] = [tag.text for tag in amenities_tags]
      infoJSON['Unaccommodations'] = [tag.text for tag in anti_amenities_tags]
    else:
      print('No button containing \"Amenities\" exists on this page.')

    # menu, if available?

    # about the business?

  # end method

Save data to CSV file.  Also saves JSON files

In [ ]:
# initial saving method.  
# Now that files are created, develop methods to modify existing files as necessary.

def save_data_to_csv(urls, fileName, mode):
  alreadySaved = os.listdir(ROOT + '/Data_Collecting/JSON files') # used to check that we do not already have a JSON saved
  with open(ROOT + '/Data_Collecting/' + fileName + '.csv', mode) as f:
    writer = csv.writer(f)
    header = ['Name', 'Address', 'City', 'State', 'Zip', 'Phone', 'Price_Range', 'Ratings', 'Review_Count', 'URL', 'Tag', 'Day', 'Hours']
    writer.writerow(header)
    counter = 0;
    for url in urls:
      counter += 1
      print('count: ' + str(counter))
      est_info = []
      get_yelp_establishment_information(url, est_info, alreadySaved)
      print()
      writer.writerow(est_info)
  f.close()


In [ ]:
save_data_to_csv(distillery_url_list, 'Distillery_Data', 'w') # or 'a' for appending to file

In [ ]:
save_data_to_csv(winery_url_list, 'Winery_Data', 'w') # or 'a' for appending to file

In [ ]:
save_data_to_csv(brewery_url_list, 'brewery_Data', 'w') # or 'a' for appending to file